<a href="https://colab.research.google.com/github/omjadhav007/PIB_to_Video/blob/main/Capstone_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
# # Load the saved model and tokenizer
# model = T5ForConditionalGeneration.from_pretrained("t5-large")
# tokenizer = T5Tokenizer.from_pretrained("t5-large")


# model.save_pretrained("/content/drive/MyDrive/t5large_model", from_pt=True)
# tokenizer.save_pretrained("/content/drive/MyDrive/t5large_model", from_pt=True)

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/t5large_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/t5large_model")

def abstractive_summarization(text):
    # Tokenize the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    l=len(text)
    # Generate summary
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    # Decode summary tokens back to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# !pip install gTTS
# !pip install pydub
# !pip install phonemizer
# !pip install moviepy
# !pip install beautifulsoup4
# !pip install urllib3
import sys
sys.path.append('/content/')
import urllib.request
import lxml
from bs4 import BeautifulSoup
import pandas as pd


theurl = "https://pib.gov.in/indexd.aspx"
thepage = urllib.request.urlopen(theurl)

soup = BeautifulSoup(thepage,'html.parser')
u=soup.find('ul',class_='release_list')
hl=[]
rel=[]
des=[]
sum=[]
lks = ["https://pib.gov.in/PressReleasePage"+i['href'][18:] for i in u.find_all('a',href=True)]
for i in lks:
    page = urllib.request.urlopen(i)
    s = BeautifulSoup(page,'html.parser')
    hl.append(s.find('h2').text.strip())
    rel.append(s.find('div',class_='ReleaseDateSubHeaddateTime').text.strip())
    z=s.find_all('p')
    d=""
    for j in z:
        d+=j.text.strip()
        if d=="":
            break
    des.append(d)
    # print(len(d))
obj={'links':lks, 'headline':hl, 'release':rel, 'description':des}
# print(d)
# df=pd.DataFrame(obj)

# for i in range(len(sum)):
#     extr.text_to_speech(obj['headline'][i][:5]+str(i+1),sum[i])

# sum=abstractive_summarization(des[0])
# print(sum)
# print(df)

# df.to_csv("newPIB_data.csv")





In [ ]:
for i in range(len(des)):
    sum.append(abstractive_summarization(des[i]))

In [ ]:
import cv2
import numpy as np
import random
from gtts import gTTS
from pydub import AudioSegment
from phonemizer import phonemize
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips, ImageClip

from phonemizer.backend.espeak.wrapper import EspeakWrapper
# EspeakWrapper.set_library('C:\Program Files\eSpeak NG\libespeak-ng.dll')


def audio_creation(file_name,text):
    tts = gTTS(text, lang='en')
    tts.save("Audios/"+file_name + '.mp3')
    audio = AudioSegment.from_mp3("Audios/"+file_name + '.mp3')
    audio_length=len(audio)
    return audio_length

def video_creation(text,file_name,audio_length):
    # phonemes = phonemize(text, backend='espeak', language='en-us',strip=True, preserve_punctuation=True)
    text=text.lower()
    text_length=len(text)
    phoneme_to_image = {
        'a': 'girl/1.png'
    }
    l=[]
    for i in range(12):
        l.append("girl/"+str(i+1)+".png")

    for i in range(26):
        s=i
        s+=ord('a')
        s=chr(s)
        index=4
        while index==4:
            index=random.randrange(12)
        phoneme_to_image[s]=l[index]
    # print(phoneme_to_image)


    # Prepare list of image clips
    image_clips = []
    phoneme_duration = audio_length/(1000*text_length)
    print(audio_length//phoneme_duration)
    for phoneme in text:
        avatar_image = l[4]
        if phoneme in phoneme_to_image:
            avatar_image =phoneme_to_image[phoneme]
        img_clip = ImageClip(avatar_image).set_duration(phoneme_duration)
        image_clips.append(img_clip)

    print(len(image_clips))
    # print(image_clips)

    # Step 4: Video Creation
    video = concatenate_videoclips(image_clips, method="compose")
    audio_clip = AudioFileClip("Audios/"+file_name+".mp3")
    video = video.set_audio(audio_clip)
    video.write_videofile("Videos/"+file_name+".mp4", fps=5)

    print("Video has been saved as '"+file_name+".mp4'")




In [ ]:
for i in range(len(sum)):
  para = sum[i]
  file_name=hl[i][:5]+str(i+1)
  aud_len=audio_creation(file_name,para)
  video_creation(para,file_name,aud_len)

278999.0
279
Moviepy - Building video Videos/version3.mp4.
MoviePy - Writing audio in version3TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Videos/version3.mp4



Moviepy - Done !
Moviepy - video ready Videos/version3.mp4
Video has been saved as 'version3.mp4'
